In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

template_path= 'assets\\template_12.jpg'
target_path = 'assets\\bikes\\bike_15.jpg'

point_match_template = cv2.imread(template_path,cv2.CV_8UC1)          # query Image
point_match_template = cv2.resize(point_match_template,(500,500))

img2 = cv2.imread(target_path)  # target Image
img2 = cv2.resize(img2,(500,500))

point_match_image = cv2.imread(target_path)
point_match_image = cv2.resize(point_match_image,(500,500))


img3 = img2.copy()
img4 = img2.copy()
# point_match_template = cv2.resize(point_match_template,(200,200))
# img2 = cv2.resize(img2,(500,500))
image_2_hsv=cv2.cvtColor(img2.copy(),cv2.COLOR_RGB2HSV)
# lower_yellow=np.array([101,96,0])
lower_yellow=np.array([100,56,0])
upper_yellow=np.array([120,255,255])

point_match_lower_yellow=np.array([100,96,0])
point_match_upper_yellow=np.array([1166,255,255])
point_match_image = cv2.inRange(point_match_image,point_match_lower_yellow,point_match_upper_yellow)
img2 = cv2.inRange(image_2_hsv,lower_yellow,upper_yellow)

# Point match using SIFT
template_kp,template_des,target_kp,target_des,good_matches = get_SIFT_match_points(point_match_template,point_match_image)

print('match points:',len(good_matches))
if len(good_matches)< 4:
    print("No Point Match")
    point_match_frame = point_match_image
else:
    point_match_frame = draw_keypoints(point_match_image.copy(),good_matches)


    point_match_frame = cv2.drawMatchesKnn(point_match_template,template_kp,point_match_image,target_kp,good_matches, None,flags = 2)


# Shape matcch
template_frame = cv2.imread('assets\\template_13.jpg',cv2.CV_8UC1)
template_frame = cv2.resize(template_frame,(200,200))
# template contours
template_contours,template_heridency = cv2.findContours(template_frame,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
template = template_contours[0]

# Initiate SIFT detector


contours, hierarchy = cv2.findContours(img2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
marea=None
for cnt in contours:
    approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
    # cv2.drawContours(img3,[cnt],0,[255,255,0],3)
    marea = cnt if marea is None or cv2.contourArea(cnt)>cv2.contourArea(marea) else marea

cv2.drawContours(img3,[marea],0,[255,255,0],3)
print('max area match rate:',cv2.matchShapes(template,marea,3,0.0))

        
min_pos = -1
min_value = 3
for i in range(len(contours)):
        # 参数3：匹配方法；参数4：opencv预留参数
    value = cv2.matchShapes(template,contours[i],3,0.0)
    if value < min_value and cv2.contourArea(contours[i]) > 100:
        min_value = value
        min_pos = i
    # 参数3为0表示绘制本条轮廓contours[min_pos]
if min_pos >=0:
    cv2.drawContours(img3,[contours[min_pos]],0,[255,0,0],3)
    cv2.drawContours(img4,[contours[min_pos]],0,[255,0,0],3)
    img4 = cv2.drawMatchesKnn(point_match_template,template_kp,img4,target_kp,good_matches, None,flags = 2)

    
    print("mp:",min_pos)

    ins = 0
    outs = 0
    for m in good_matches:
        x,y=target_kp[m[0].trainIdx].pt
        x= int(x)
        y=int(y)
        result = cv2.pointPolygonTest(contours[min_pos], (x,y), True) 
        print(result)
        result = cv2.pointPolygonTest(contours[min_pos], (x,y), False) 

        if result == 1 or result == 0:
            # print('inside')
            ins+=1
        else:
            # print('outside')
            outs+=1
    print('ins:',ins)
    print('outs',outs)
    if (ins+outs)>0:
        print('in rate:',ins/(ins+outs))
    # point_match_frame = cv2.polylines(point_match_frame, [np.int32(dst)], True, (0,0,255),3, cv2.LINE_AA)




cv2.imshow('result',img3)
cv2.imshow('mask',img2)
cv2.imshow('template',template_frame)
cv2.imshow('save_img',img4)

cv2.imshow('point_match',point_match_frame)
cv2.waitKey(0)
cv2.destroyAllWindows()
# plt.savefig('matche_2.png', dpi = 300)